# BipedalWalker-v2 with PPO, Vectorized Environment
In this notebook, you will implement a PPO agent with OpenAI Gym's BipedalWalker-v2 environment.

### 1. Create Vectorized Environment

In [2]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

from  collections  import deque
import time
from model import Policy
from ppo import ppo_agent
from storage import RolloutStorage
from utils import get_render_func, get_vec_normalize
from envs import make_vec_envs
from parallelEnv import parallelEnv
import matplotlib.pyplot as plt
%matplotlib inline

print('gym version: ', gym.__version__)
print('torch version: ', torch.__version__)

seed = 0 
gamma=0.99
num_processes=16 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device: ', device)

envs = parallelEnv('BipedalWalker-v3', n=num_processes, seed=seed)

## make_vec_envs -cannot find context for 'forkserver'
## forkserver is only available in Python 3.4+ and only on some Unix platforms (not on Windows).
## envs = make_vec_envs('BipedalWalker-v2', \
##                    seed + 1000, num_processes,
##                    None, None, False, device='cpu', allow_early_resets=False)

max_steps = envs.max_steps
print('max_steps: ', max_steps)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)


gym version:  0.19.0
torch version:  1.10.2+cpu
device:  cpu
max_steps:  1600


### 2. Instantiate Model, Agent and Storage

Initialize the Policy (model MLPBase), PPO Agent and Rollout Storage.

In [3]:
## model Policy uses MLPBase
policy = Policy(envs.observation_space.shape, envs.action_space,\
        base_kwargs={'recurrent': False})

policy.to(device)

agent = ppo_agent(actor_critic=policy, ppo_epoch=16, num_mini_batch=16,\
                lr=0.001, eps=1e-5, max_grad_norm=0.5)

rollouts = RolloutStorage(num_steps=max_steps, num_processes=num_processes, \
                        obs_shape=envs.observation_space.shape, action_space=envs.action_space, \
                        recurrent_hidden_state_size=policy.recurrent_hidden_state_size)

obs = envs.reset()
print('type obs: ', type(obs), ', shape obs: ', obs.shape)
obs_t = torch.tensor(obs)
print('type obs_t: ', type(obs_t), ', shape obs_t: ', obs_t.shape)

rollouts.obs[0].copy_(obs_t)
rollouts.to(device)


type obs:  <class 'numpy.ndarray'> , shape obs:  (16, 24)
type obs_t:  <class 'torch.Tensor'> , shape obs_t:  torch.Size([16, 24])


### 3.Save model function

In [4]:
def save(model, directory, filename, suffix):
    torch.save(model.base.actor.state_dict(), '%s/%s_actor_%s.pth' % (directory, filename, suffix))
    torch.save(model.base.critic.state_dict(), '%s/%s_critic_%s.pth' % (directory, filename, suffix))
    torch.save(model.base.critic_linear.state_dict(), '%s/%s_critic_linear_%s.pth' % (directory, filename, suffix))
    
limits = [-300, -160, -100, -70, -50, 0, 20, 30, 40, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]

def return_suffix(j):
    suf = '0'
    for i in range(len(limits)-1):
        if j > limits[i] and j < limits[i+1]:
            suf = str(limits[i+1])
            break
        
        i_last = len(limits)-1    
        if  j > limits[i_last]:
            suf = str(limits[i_last])
            break
    return suf      

### 4. Train the Agent  with Vectorized Environment

In [ ]:
# num_updates=1000000
# gamma = 0.99
# tau=0.95
# save_interval=30
# log_interval= 1 

# def ppo_vec_env_train(envs, agent, policy, num_processes, num_steps, rollouts):
    
#     time_start = time.time()
    
#     n=len(envs.ps)    
#     envs.reset()
    
#     # start all parallel agents
#     print('Number of agents: ', n)
#     envs.step([[1]*4]*n)
    
#     indices = []
#     for i  in range(n):
#         indices.append(i)
     
#     s = 0
    
#     scores_deque = deque(maxlen=100)
#     scores_array = []
#     avg_scores_array = []    

#     for i_episode in range(num_updates):
        
#         total_reward = np.zeros(n)
#         timestep = 0
        
#         for timestep in range(num_steps):

#             with torch.no_grad():
#                 value, actions, action_log_prob, recurrent_hidden_states = \
#                    policy.act(
#                         rollouts.obs[timestep],
#                         rollouts.recurrent_hidden_states[timestep],
#                         rollouts.masks[timestep])
                
#             obs, rewards, done, infos = envs.step(actions.cpu().detach().numpy())
            
#             total_reward += rewards  ## this is the list by agents
                        
#             # If done then clean the history of observations.
#             masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
#             obs_t = torch.tensor(obs)
#             ## Add one dimnesion to tensor, otherwise does not work
#             ## This is (unsqueeze(1)) solution for:
#             ## RuntimeError: The expanded size of the tensor (1) must match the existing size...
#             rewards_t = torch.tensor(rewards).unsqueeze(1)
#             rollouts.insert(obs_t, recurrent_hidden_states, actions, action_log_prob, \
#                 value, rewards_t, masks)
                    
#         avg_total_reward = np.mean(total_reward)
#         scores_deque.append(avg_total_reward)
#         scores_array.append(avg_total_reward)
                
#         with torch.no_grad():
#             next_value = policy.get_value(rollouts.obs[-1],
#                             rollouts.recurrent_hidden_states[-1],
#                             rollouts.masks[-1]).detach()

#         rollouts.compute_returns(next_value, gamma, tau)

#         agent.update(rollouts)

#         rollouts.after_update()
        
#         avg_score = np.mean(scores_deque)
#         avg_scores_array.append(avg_score)

#         if i_episode > 0 and i_episode % save_interval == 0:
#             print('Saving model, i_episode: ', i_episode, '\n')
#             suf = return_suffix(avg_score)
#             save(policy, 'dir_save', 'we0', suf)
#             #save_venv(policy, 'dir_save_VecEnv', 'final')

        
#         if i_episode % log_interval == 0 and len(scores_deque) > 1:            
#             prev_s = s
#             s = (int)(time.time() - time_start)
#             t_del = s - prev_s
#             print('Ep. {}, Timesteps {}, Score.Agents: {:.2f}, Avg.Score: {:.2f}, Time: {:02}:{:02}:{:02}, \
# Interval: {:02}:{:02}'\
#                    .format(i_episode, timestep+1, \
#                         avg_total_reward, avg_score, s//3600, s%3600//60, s%60, t_del%3600//60, t_del%60)) 
    
#         if len(scores_deque) == 100 and np.mean(scores_deque) > 300.5:   
#         # if np.mean(scores_deque) > 20:   
#             print('Environment solved with Average Score: ',  np.mean(scores_deque) )
#             break
    
    
#     return scores_array, avg_scores_array
            

In [10]:
import time
from collections import deque
import numpy as np
import torch

def ppo_vec_env_train(envs, agent, policy, num_processes, num_steps, rollouts, num_updates=1000000, gamma=0.99, tau=0.95, save_interval=30, log_interval=1):
    time_start = time.time()
    
    n = len(envs.ps)    
    envs.reset()
    
    print('Number of agents: ', n)
    envs.step([[1]*4]*n)
    
    indices = []
    for i in range(n):
        indices.append(i)
     
    s = 0
    scores_deque = deque(maxlen=100)
    scores_array = []
    avg_scores_array = []    

    for i_episode in range(num_updates):
        total_reward = np.zeros(n)
        timestep = 0
                
        for timestep in range(num_steps):
            if render and timestep == 0:
                print(f"Rendering episode {i_episode}")

            with torch.no_grad():
                value, actions, action_log_prob, recurrent_hidden_states = policy.act(
                    rollouts.obs[timestep],
                    rollouts.recurrent_hidden_states[timestep],
                    rollouts.masks[timestep]
                )
                
            obs, rewards, done, infos = envs.step(actions.cpu().detach().numpy())
            
            envs.render(mode='human')  # Render the environment, 'human' mode usually shows a window

            total_reward += rewards  ## this is the list by agents
                        
            masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
            obs_t = torch.tensor(obs)
            rewards_t = torch.tensor(rewards).unsqueeze(1)
            rollouts.insert(obs_t, recurrent_hidden_states, actions, action_log_prob, value, rewards_t, masks)
                    
        avg_total_reward = np.mean(total_reward)
        scores_deque.append(avg_total_reward)
        scores_array.append(avg_total_reward)
                
        with torch.no_grad():
            next_value = policy.get_value(rollouts.obs[-1],
                                          rollouts.recurrent_hidden_states[-1],
                                          rollouts.masks[-1]).detach()

        rollouts.compute_returns(next_value, gamma, tau)

        agent.update(rollouts)

        rollouts.after_update()
        
        avg_score = np.mean(scores_deque)
        avg_scores_array.append(avg_score)

        if i_episode > 0 and i_episode % save_interval == 0:
            print('Saving model, i_episode: ', i_episode, '\n')
            suf = return_suffix(avg_score)
            save(policy, 'dir_save', 'we0', suf)
        
        if i_episode % log_interval == 0 and len(scores_deque) > 1:            
            prev_s = s
            s = (int)(time.time() - time_start)
            t_del = s - prev_s
            print(f'Ep. {i_episode}, Timesteps {timestep+1}, Score.Agents: {avg_total_reward:.2f}, Avg.Score: {avg_score:.2f}, Time: {s//3600:02}:{s%3600//60:02}:{s%60:02}, Interval: {t_del%3600//60:02}:{t_del%60:02}') 
    
        if len(scores_deque) == 100 and np.mean(scores_deque) > 300.5:   
            print('Environment solved with Average Score: ',  np.mean(scores_deque) )
            break
    
    return scores_array, avg_scores_array


In [11]:
scores, avg_scores = ppo_vec_env_train(envs, agent, policy, num_processes, max_steps, rollouts)


Number of agents:  16
Rendering episode 0
Ep. 1, Timesteps 1600, Score.Agents: 7.82, Avg.Score: -13.02, Time: 00:00:23, Interval: 00:23
Rendering episode 2
Ep. 2, Timesteps 1600, Score.Agents: 4.53, Avg.Score: -7.17, Time: 00:00:33, Interval: 00:10
Ep. 3, Timesteps 1600, Score.Agents: 20.53, Avg.Score: -0.25, Time: 00:00:44, Interval: 00:11
Rendering episode 4
Ep. 4, Timesteps 1600, Score.Agents: 19.99, Avg.Score: 3.80, Time: 00:00:55, Interval: 00:11
Ep. 5, Timesteps 1600, Score.Agents: -2.91, Avg.Score: 2.68, Time: 00:01:06, Interval: 00:11
Rendering episode 6


KeyboardInterrupt: 

In [ ]:
save(model=policy,directory='dir_save',filename='we0',suffix='final')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

print('length of scores: ', len(scores), ', len of avg_scores: ', len(avg_scores))

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores, label="Score")
plt.plot(np.arange(1, len(avg_scores)+1), avg_scores, label="Avg on 100 episodes")
plt.legend(bbox_to_anchor=(1.05, 1)) 
plt.ylabel('Score')
plt.xlabel('Episodes #')
plt.show()

In [ ]:
#--------------- make_vec_envs ----------------
## we continue with the same model, model Policy uses MLPBase, but with new environment env_venv

device = torch.device("cpu")
print('device: ', device)

seed = 0 
num_processes=1  

env_venv = make_vec_envs('BipedalWalker-v2', \
                    seed + 1000, num_processes,
                    None, None, False, device='cpu', allow_early_resets=False)

policy = policy.to(device)

print('env_venv.observation_space.shape: ', env_venv.observation_space.shape, \
      ', len(obs_shape): ', len(env_venv.observation_space.shape))
print('env_venv.action_space: ',  env_venv.action_space, \
      ', action_space.shape[0]: ', env_venv.action_space.shape[0])


In [ ]:
## No CUDA, only CPU

def play_VecEnv(env, model, num_episodes):

    obs = env.reset()
    obs = torch.Tensor(obs)
    obs = obs.float()
        
    recurrent_hidden_states = torch.zeros(1, model.recurrent_hidden_state_size)
    
    masks = torch.zeros(1, 1)
    
    scores_deque = deque(maxlen=100)

    render_func = get_render_func(env)
        
    for i_episode in range(1, num_episodes+1):     

        time_start = time.time()
        total_reward = np.zeros(num_processes)
        timestep = 0

        while True:
        
            with torch.no_grad():
                value, action, _, recurrent_hidden_states = \
                    model.act(obs, recurrent_hidden_states, masks, \
                    deterministic=False) # obs = state

            render_func()
            
            obs, reward, done, _ = env.step(action)
            obs = torch.Tensor(obs)
            obs = obs.float()

            reward = reward.detach().numpy()
            
            masks.fill_(0.0 if done else 1.0)
            
            total_reward += reward[0]
        
            #if timestep < 800:
            #    print('timestep: ', timestep, 'reward: ', reward[0])
            
            timestep += 1
            
            if timestep + 1 == 1600: ##   envs.max_steps:
                break

        s = (int)(time.time() - time_start)
        
        scores_deque.append(total_reward[0])
        
        avg_score = np.mean(scores_deque)
                    
        print('Episode {} \tScore: {:.2f}, Avg.Score: {:.2f}, \tTime: {:02}:{:02}:{:02}'\
                  .format(i_episode, total_reward[0], avg_score,  s//3600, s%3600//60, s%60))  


In [ ]:
play_VecEnv(env=env_venv, model=policy, num_episodes=5)

In [ ]:
#env_venv.close()
#obs = env_venv.reset()
#obs.close()
#env_venv.close()

In [ ]:
#def load(model):
#    model.base.actor.load_state_dict(torch.load('dir_save_VecEnv\we0_actor_final.pth'))
#    model.base.critic.load_state_dict(torch.load('dir_save_VecEnv\we0_critic_final.pth'))
#    model.base.critic_linear.load_state_dict(torch.load('dir_save_VecEnv\we0_critic_linear_final.pth'))

#def load_venv(model):
#    model.base, ob_rms = \
#            torch.load('dir_save_VecEnv\weights_venv_final.pth')


# load(model=policy)    
# load_venv(model=policy)
# play_episode_parallelEnv(envs=envs, model=policy)